In [23]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [24]:
train_data = pd.DataFrame(json_data)
train_data = train_data[['tags', 'songs', 'like_cnt']]
train_data.head()

,tags,songs,like_cnt
0,[락],"[525514, 129701, 383374, 562083, 297861, 13954...",71
1,"[추억, 회상]","[432406, 675945, 497066, 120377, 389529, 24427...",1
2,"[까페, 잔잔한]","[83116, 276692, 166267, 186301, 354465, 256598...",17
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...","[394031, 195524, 540149, 287984, 440773, 10033...",33
4,[댄스],"[159327, 553610, 5130, 645103, 294435, 100657,...",9


In [25]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [28]:
song_all = pd.DataFrame(json_data)
song_all = song_all[['song_gn_dtl_gnr_basket', 'artist_id_basket', 
                     'song_name', 'song_gn_gnr_basket', 'artist_name_basket', 'id']]
song_all.head()

,song_gn_dtl_gnr_basket,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",[560160],그남자 그여자,[GN1800],[Jude Law],4
